**Classificação pelo método voting**

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [6]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.912


Vemos que com os resultados das classificações dos diversos testes agregados, a acurácia do VotingClassifier ficou maior se comparada com a dos testes individualmente. Este tipo classificador ensemble funciona da seguinte forma, diferentes tipos de algoritmos de machine learning são treinados individualmente. Para fazer a previsão, todos eles serão usados a partir do seu próprio resultado de teste, em seguida são compados os seus resultados, a classe que obteve o maior número de previsões será a escolhida como resultado da previsão do VotingClassifier. Isso porque especificamos o método **hard**

In [8]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [9]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.92


Outra maneira que podemos usar o VotingClassifier, além da rígida(hard), é substituindo-o por **soft**. Desta maneira a escolha pela classe prevista é a partir da que tiver a maior probabilidade média entre todos os classificadores individuais. Ele consegue um desempenho maior que o **hard**, pois da mais peso para votos mais confiantes. O problema deste classificador é que os modelos usados precisam ter o método **predict_proba()** que tem a saída das probalidade das classes estarem certas.

No caso do SVM, ele não possui o método específico, porém se marcado como **probability=True**, ele faz com que SVC use validação cruzada para fazer a probabilidade das classes.

**Bagging e Pasting**

In [36]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)

bag_clf.fit(X_train, y_train)

y_pred = bag_clf.predict(X_test)

In [37]:
print(bag_clf)
print("Accuracy = ", accuracy_score(y_test, y_pred))

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1)
Accuracy =  0.92


No código em questão foram usadas 500 Árvores de decisão, o tamanho das amostras é 100, n_jobs=-1 diz que todos os núcleos disponíveis da CPU serão usados.

O parâmetro bootstrap informa se o ensemble será bagging ou pasting, caso seja bagging, bootstrap será **True** dizendo ao modelo que haverá reposição das intâncias já usadas. Quando bootstrap=**False**, informa que não haverá reposição das instâncias.

In [38]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=False, n_jobs=-1)

bag_clf.fit(X_train, y_train)

y_pred = bag_clf.predict(X_test)

In [39]:
print(bag_clf)
print("Accuracy = ", accuracy_score(y_test, y_pred))

BaggingClassifier(bootstrap=False, estimator=DecisionTreeClassifier(),
                  max_samples=100, n_estimators=500, n_jobs=-1)
Accuracy =  0.92


In [40]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=500,
                  n_jobs=-1, oob_score=True)

In [41]:
bag_clf.oob_score_

0.8986666666666666

In [42]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.92

Como Bagging utiliza instâncias de forma aleatória, algumas instâncias podem ser usadas mais de uma vez, enquanto outras não serão usadas. Com isso em vez de usar uma validação cruzada para avaliar o modelo, podemos utilizar o parâmetro oob_score=True, onde o modelo usará essas instâncias não utilizadas para fazer uma autoavaliação. No exemplo em questão a sua autoavaliação deu aproximadamente 90%, enquanto o modelo com o teste dá 92%, sendo assim podemos dizer com o modelo está com uma boa generalização.

**Random Forest**

In [44]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)

rnd_clf.fit(X_train, y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1)

In [47]:
y_pred_rf = rnd_clf.predict(X_test)

print(accuracy_score(y_test, y_pred_rf))

0.904


Conhecendo a importância das features

In [50]:
from sklearn.datasets import load_iris

iris = load_iris()

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.08947929803113068
sepal width (cm) 0.02275603171638201
petal length (cm) 0.4401272047447944
petal width (cm) 0.44763746550769296
